In [14]:
import netCDF4 as nc
import numpy as np
from skimage.transform import downscale_local_mean

In [11]:
myPath = '/Users/leguillou/WORK/Developpement/'
dir_data =  myPath + "DATA/eNATL60/merged_outputs/hourly/"
dir_output = myPath + "DATA/eNATL60/merged_outputs/hourly/"
filename_data = "eNATL60_GULFSTREAM_SSH.1h_y2009m07d01_y2010m06d30.nc"

In [35]:
fact = 3

res = int(60/fact)

file_output = filename_data[:-3] + '_degraded' + str(res) + '.nc'

print(file_output)


eNATL60_GULFSTREAM_SSH.1h_y2009m07d01_y2010m06d30_degraded20.nc


In [8]:
ncin = nc.Dataset(dir_data+filename_data)
times = np.float64(ncin.variables['time_counter'][:]) 
lon = np.float64(ncin.variables['nav_lon'][:,:]) % 360
lat = np.float64(ncin.variables['nav_lat'][:,:]) 
ssh = ncin.variables['sossheig'][:,:,:]


In [9]:
dlon = lon[0,1] - lon[0,0]
dlat = lat[1,0] - lat[0,0]
lon_down = lon[::fact,::fact] + (fact-1)/2*dlon
lat_down = lat[::fact,::fact] + (fact-1)/2*dlat
NY,NX = lon_down.shape
NT = times.size


In [39]:
ncout = nc.Dataset(dir_output + file_output, 'w', format='NETCDF4_CLASSIC')
ncout.createDimension('time', NT)
ncout.createDimension('y', NY)
ncout.createDimension('x', NX)
nctime = ncout.createVariable('time', 'f', ('time',))
nctime.units = ncin.variables['time_counter'].units
nclon = ncout.createVariable('nav_lon', 'f', ('y','x'))
nclat = ncout.createVariable('nav_lat', 'f', ('y','x'))
nctime[:] = times
nclon[:,:] = lon_down
nclat[:,:] = lat_down
ncssh = ncout.createVariable('degraded_sossheig', 'f', ('time','y','x'))
for t in range(NT):
    ncssh[t,:,:] = downscale_local_mean(ssh[t],(fact,fact))
ncout.close()
